In [ ]:
import pandas as pd
import statsmodels.api as sm

In [2]:
# Load the data
voyage_options = pd.read_csv('data/voyage_options.csv')
vendor_costs = pd.read_csv('data/vendor_costs_654.csv')

# Display basic info about the data
print("Voyage Options Data:")
print(voyage_options.head())
print("\nVendor Costs Data:")
print(vendor_costs.head())

Voyage Options Data:
     entertainment                                    dining  \
0  East Coast Folk  Maine Lobster Nova Scotia Seafood Buffet   
1  East Coast Folk  Maine Lobster Nova Scotia Seafood Buffet   
2  East Coast Folk  Maine Lobster Nova Scotia Seafood Buffet   
3  East Coast Folk  Maine Lobster Nova Scotia Seafood Buffet   
4  East Coast Folk  Maine Lobster Nova Scotia Seafood Buffet   

                 cabin            amenities  cocktail_credits  avg_rating  
0  Cozy with Fireplace    Top Deck Hot Tubs                 1    8.313079  
1  Cozy with Fireplace    Top Deck Hot Tubs                 2    9.435885  
2  Cozy with Fireplace    Top Deck Hot Tubs                 3    8.540985  
3  Cozy with Fireplace  Winter Wellness Spa                 1    8.415454  
4  Cozy with Fireplace  Winter Wellness Spa                 2    9.241294  

Vendor Costs Data:
                                       Item  Item Category  \
0                           East Coast Folk  entertainme

In [ ]:
# Prepare data for conjoint analysis
features = ['entertainment', 'dining', 'cabin', 'amenities', 'cocktail_credits']
X = pd.get_dummies(voyage_options[features], drop_first=False)

# Convert to numpy arrays to avoid dtype issues
X = X.astype(float)  
y = voyage_options['avg_rating'].values  

# Add constant term
X = sm.add_constant(X)

# Fit linear regression model for conjoint analysis
model = sm.OLS(y, X).fit()

# Get part-worth utilities (coefficients)
part_worth = model.params[1:]  # Exclude the constant term
print("\nPart-worth utilities (coefficients):")
print(part_worth)

# Calculate importance of each attribute
importance = {}
for feature in features:
    col_mask = part_worth.index.str.startswith(feature)
    importance[feature] = (part_worth[col_mask].max() - part_worth[col_mask].min()) / part_worth.abs().sum() * 100

print("\nAttribute Importance (%):")
print(importance)


Part-worth utilities (coefficients):
cocktail_credits                                  -0.266540
entertainment_Acoustic Showcase                    0.838775
entertainment_DJ_Dance                             0.140997
entertainment_East Coast Folk                      1.283075
entertainment_Jazz Blues                           1.194150
dining_Casual Fireside Grill with Hot Cider        0.608897
dining_Coastal Tapas & Tasting Stations            0.901833
dining_Formal Multi-Course Plated                 -0.159852
dining_Maine Lobster Nova Scotia Seafood Buffet    2.106119
cabin_Cozy with Fireplace                          0.890427
cabin_Modern Minimalist                            0.440748
cabin_Nautical Chic                                1.202862
cabin_Romantic Escape                              0.922960
amenities_Fireside Live Music                      0.825440
amenities_Top Deck Hot Tubs                        1.728434
amenities_Winter Wellness Spa                      0.903123
dt

In [9]:
# Find the best combination within budget
# First create a cost mapping dictionary
cost_map = {}
for _, row in vendor_costs.iterrows():
    if row['Item Category'] == 'cocktail':
        key = f"{row['Item'].split()[0]} cocktail tickets per passenger"
    else:
        # Handle special cases where names don't match exactly
        if row['Item'] == 'DJ Dance':  
            key = 'DJ_Dance'
        else:
            key = row['Item']
    cost_map[key] = row['Cost Per Passenger (Dollars)']

# Evaluate all possible combinations
best_rating = 0
best_combo = None
best_cost = 0

# Iterate through all unique combinations in the dataset
for _, row in voyage_options.drop_duplicates(subset=features).iterrows():
    # Calculate total cost
    cost = (cost_map[row['entertainment']] + 
            cost_map[row['dining']] + 
            cost_map[row['cabin']] + 
            cost_map[row['amenities']] + 
            cost_map[f"{row['cocktail_credits']} cocktail tickets per passenger"])
        
    # Check if within budget
    if cost <= 75:
        # Get predicted rating
        rating = row['avg_rating']
        if rating > best_rating:
            best_rating = rating
            best_combo = row[features]
            best_cost = cost
    continue

# Display the best combination
print("\nBest Combination Within Budget:")
print(best_combo)
print(f"Predicted Rating: {best_rating:.2f}")
print(f"Total Cost per Passenger: ${best_cost:.2f}")


Best Combination Within Budget:
entertainment                          Acoustic Showcase
dining              Casual Fireside Grill with Hot Cider
cabin                                    Romantic Escape
amenities                              Top Deck Hot Tubs
cocktail_credits                                       3
Name: 416, dtype: object
Predicted Rating: 9.98
Total Cost per Passenger: $66.75


In [10]:
# Get the top 3 combinations within budget
top_combinations = []
for _, row in voyage_options.sort_values('avg_rating', ascending=False).iterrows():
    cost = (cost_map[row['entertainment']] + 
            cost_map[row['dining']] + 
            cost_map[row['cabin']] + 
            cost_map[row['amenities']] + 
            cost_map[f"{row['cocktail_credits']} cocktail tickets per passenger"])
    
    if cost <= 75:
        combo_info = {
            'entertainment': row['entertainment'],
            'dining': row['dining'],
            'cabin': row['cabin'],
            'amenities': row['amenities'],
            'cocktail_credits': row['cocktail_credits'],
            'rating': row['avg_rating'],
            'cost': cost
        }
        top_combinations.append(combo_info)
        if len(top_combinations) >= 3:
            break

print("\nTop 3 Combinations Within Budget:")
for i, combo in enumerate(top_combinations, 1):
    print(f"\nOption {i}:")
    print(f"Entertainment: {combo['entertainment']}")
    print(f"Dining: {combo['dining']}")
    print(f"Cabin: {combo['cabin']}")
    print(f"Amenities: {combo['amenities']}")
    print(f"Cocktail Credits: {combo['cocktail_credits']}")
    print(f"Rating: {combo['rating']:.2f}")
    print(f"Cost: ${combo['cost']:.2f}")


Top 3 Combinations Within Budget:

Option 1:
Entertainment: Acoustic Showcase
Dining: Casual Fireside Grill with Hot Cider
Cabin: Romantic Escape
Amenities: Top Deck Hot Tubs
Cocktail Credits: 3
Rating: 9.98
Cost: $66.75

Option 2:
Entertainment: Jazz Blues
Dining: Casual Fireside Grill with Hot Cider
Cabin: Cozy with Fireplace
Amenities: Top Deck Hot Tubs
Cocktail Credits: 1
Rating: 9.95
Cost: $62.75

Option 3:
Entertainment: DJ_Dance
Dining: Maine Lobster Nova Scotia Seafood Buffet
Cabin: Nautical Chic
Amenities: Top Deck Hot Tubs
Cocktail Credits: 2
Rating: 9.94
Cost: $69.50


After carefully analyzing passenger ratings and cost data, we found several great  patterns for Lobsterland's voyage packages. The conjoint analysis reveals that dining experience carries the most weight in determining overall satisfaction (15.7% importance), followed by entertainment (7.9%) and amenities (6.3%). Interestingly, the number of cocktail credits showed negligible impact on ratings, though it does affect costs.

The standout recommendation is the Best Combination (Option 1), what I'd call the "Premium Experience" at $66.75 per passenger. This combines the Acoustic Showcase entertainment with the Casual Fireside Grill dining option, Romantic Escape cabins, and Top Deck Hot Tubs. It achieves the highest predicted rating of 9.98 while staying well under budget. The acoustic entertainment provides sophisticated ambiance without the higher cost of jazz bands, while the fireside grill creates a cozy social atmosphere that guests clearly love based on the ratings.

For more budget-conscious options, Option 2 at $62.75 offers excellent value. Switching to Jazz Blues entertainment and Cozy Fireplace cabins maintains a 9.95 rating while saving $4 per passenger. This could be positioned as our "Standard" offering. Meanwhile, Option 3 at $69.50 caters specifically to seafood lovers with the Maine Lobster buffet, making it our "Gourmet" option.

What's particularly interesting is how certain combinations outperform expectations. The Top Deck Hot Tubs consistently appear in top-rated packages, suggesting guests highly value this unique experience of stargazing while soaking. The Romantic Escape cabins also punch above their weight in terms of cost-to-satisfaction ratio.

I'd recommend focusing on Option 1 as the flagship offering, with Option 2 and 3 as alternatives. And we could further enhance the experience by adding small touches like live acoustic music near the hot tubs or complimentary photo services for romantic cabins, relatively low-cost additions that could boost perceived value. The cocktail credits could potentially be reduced from 3 to 2 per passenger with minimal impact on satisfaction, yielding additional savings.

All these recommendations balance exceptional guest experiences with smart cost management, and all while staying well below the $75 per passenger threshold. Because the data clearly shows that we don't need to spend to the maximum budget to achieve top-tier ratings, the key is in the strategic combination of elements.